In [17]:
##################
# Question 1
##################


import pandas as pd
import numpy as np

# reading the CSV file
df = pd.read_csv(r"mnist_train.csv")
 
# displaying the contents of the CSV file
x = df['label'].tolist()
Y = np.array(x)
# print("Y","=",Y)  # getting the output from the provided data 
c = df.drop(['label'], axis=1) #dropping the label axis from the main data and getting the remaining array as x
X = (c.to_numpy())
# print("X","=",X)

print(np.shape(X))
print(np.shape(Y))

(60000, 784)
(60000,)


In [3]:
class MLP:
    ############      Initializing all the parameters with the help of __init__() function.
    def __init__(self, network, activation_function, derivative_activation_function, loss_func, loss_func_derivative,learning_rate, epoch, diff_cost, Ypred, init_weights, init_bias, optimizer, beta):
        self.network = network
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.activation_function = activation_function
        self.activation_derivative_function = derivative_activation_function
        self.weights = init_weights
        self.bias = init_bias
        self.activation = []
        self.last_layer_derivative = []
        self.loss_func = loss_func
        self.loss_func_derivative = loss_func_derivative
        self.diff_cost = diff_cost
        self.Ypred = Ypred
        self.optimizer = optimizer
        self.beta = beta
        self.weight_error_list = []
        self.bias_error_list = []
        for i in range(len(self.network)):
            self.weight_error_list.append([])
            self.bias_error_list.append([])
    
####### Creating the forward pass algorithm.
    def forward_pass(self,X):
        self.activation = []
        A = X
        print(A.size)
        self.activation.append(A)
        for i in range(len(self.weights)-1):
            Z = np.dot(A, self.weights[i]) + self.bias[i]
            print(self.weights[i])
            A = self.activation_function(Z) 
            self.activation.append(A)
        Z = np.dot(self.activation[-1], self.weights[-1]) + self.bias[-1]
        A = self.activation_function(Z)
        self.activation.append(A)

########## Creating the algorithm for gradient descent
    def gradient_descent(self, last_weights, last_bias,weight_error, bias_error, index):
        if self.optimizer == 'momentum':
            n = int(len(self.weight_error_list[index]))
            weight_velocity = 0
            bias_velocity = 0
            for i in range(0, len(self.weight_error_list[index])):
                weight_velocity += (self.beta ** (n - i)) * self.weight_error_list[index][i]  
                bias_velocity += (self.beta ** (n - i)) * self.bias_error_list[index][i]      
            weight_velocity = (1 - self.beta)*weight_velocity
            bias_velocity = (1 - self.beta)*bias_velocity
            new_weights = last_weights - self.learning_rate * weight_velocity
            new_bias = last_bias - self.learning_rate * bias_velocity  
        elif self.optimizer == 'simple':
            new_weights = last_weights - self.learning_rate * weight_error
            new_bias = last_bias - self.learning_rate * bias_error
        return new_weights, new_bias


################ Creating the algorithm for the backward pass.
    def backward_pass(self, X, Y):
        next_layer_error = self.loss_func_derivative(Y, self.activation[-1]) 

        for i in reversed(range(0, (len(self.activation) - 1))):
            #print(i, self.activation[i].shape)
            out_fac = np.multiply(next_layer_error, self.activation_derivative_function(self.activation[i+1]))
            weight_error = (self.activation[i].T) @ out_fac
            bias_error = np.sum(out_fac, axis = 0)
            #print(weight_error.shape)
            next_layer_error =  out_fac @ self.weights[i].T
            self.weight_error_list[i].append(weight_error)
            self.bias_error_list[i].append(bias_error)
            #print(i)
            self.weights[i], self.bias[i] = self.gradient_descent(self.weights[i], self.bias[i], weight_error, bias_error, i)
            #self.weights[i] -= self.learning_rate * weight_error
            #self.bias[i] -= self.learning_rate * bias_error
        

############## Function for calculating the cost
    def cost_calc(self, Ypred, Yactual):
        return self.loss_func(Yactual, Ypred)


    def predict(self, X_given):
        self.forward_pass(X_given)
        predict_list = []
        for i in self.activation[-1]:
            predict_list.append(np.argmax(i))
        return predict_list

    def flattening(self, Y):
        a, b = Y.shape[0], self.network[-1]
        new_Y = np.zeros((a,b))
        for i in range(Y.shape[0]):
            new_Y[i, Y[i]] = 1 

        return new_Y

        
    def accuracy(self, X_test, Y_test):
        Y_pred = np.array(self.predict(X_test))
        correct_results = 0
        for i in range(Y_test.shape[0]):
            if Y_pred[i] == Y_test[i]:
                correct_results += 1
        return correct_results/Y_test.shape[0]




########### Algorithm for training means 
    def train(self, X, Y, bound):
        i = 0
        Y = self.flattening(Y)
        
        while i < self.epoch and self.diff_cost > bound:
            prev_cost = self.cost_calc(self.Ypred, Y)
            self.forward_pass(X)
            self.backward_pass(X, Y)
            self.Ypred = self.activation[-1]
            i += 1
            #number_of_times += 1
            self.diff_cost = abs(self.cost_calc(self.Ypred, Y) - prev_cost)
            print("Diff in Cost in the run ", self.diff_cost)
        print(i)
        


In [4]:
#Sigmoid funstion
def sigmoid(x):
    return 1/(np.exp(-x)+1)    

#derivative of sigmoid
def sigmoid_gradient(x):
    return (np.exp(-x))/((np.exp(-x)+1)**2)

# function for categorical cross entropy
def categorical_cross_entropy(y, y_hat):
    return -np.sum(y * np.log(y_hat + 1e-9))/y.shape[0]

 
# function for categorical cross entropy
def categorical_cross_entropy_prime(y, y_hat):
    return -y/((y_hat+ 1e-9)*(y.shape[0]))

# ReLu activation function and its gradient
def relu(z):
    return np.maximum(0, z)

# gradient for relu function
def relu_gradient(z):
    return np.where(z > 0, 1, 0)




In [18]:
X_train = X[:int(X.shape[0]*0.8), :]
Y_train = Y[:int(Y.shape[0]*0.8)]
X_test = X[int(X.shape[0]*0.8):, :]
Y_test = Y[int(Y.shape[0]*0.8):]
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


(48000, 784)
(48000,)
(12000, 784)
(12000,)


In [6]:
weights = []
bias = []
network = [784, 10]

def xavier_init(in_dim, out_dim):
    xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
    return np.random.normal(0, xavier_stddev, (in_dim, out_dim))

for i in range(len(network)-1):
    weights.append(xavier_init(network[i], network[i+1]))

for j in range(len(network)-1):
    bias.append(xavier_init(1, network[j+1]))

len(weights)


1

In [7]:
model = MLP([784, 10], relu, relu_gradient, categorical_cross_entropy,categorical_cross_entropy_prime, 0.1, 100, 100, np.ones((Y_train.shape[0], 10)), weights, bias, 'momentum',0.9)

In [8]:
model.epoch = 100
model.train(X_train, Y_train, 1e-16)

37632000
Diff in Cost in the run  11.24624495778546
37632000
Diff in Cost in the run  9.524696739292802
37632000
Diff in Cost in the run  3.3091588239475724
37632000
Diff in Cost in the run  1.5361051392276523
37632000
Diff in Cost in the run  0.7532980716955833
37632000
Diff in Cost in the run  0.2998808674446609
37632000
Diff in Cost in the run  0.40432500000674754
37632000
Diff in Cost in the run  1.0290124860407142
37632000
Diff in Cost in the run  1.2398564122586446
37632000
Diff in Cost in the run  0.23743849555323582
37632000
Diff in Cost in the run  0.11551533852490081
37632000
Diff in Cost in the run  0.08542626318694069
37632000
Diff in Cost in the run  0.07010191121815712
37632000
Diff in Cost in the run  0.05916317027118634
37632000
Diff in Cost in the run  0.05079318365151764
37632000
Diff in Cost in the run  0.044140813924285816
37632000
Diff in Cost in the run  0.03871964003141759
37632000
Diff in Cost in the run  0.034222036851815574
37632000
Diff in Cost in the run  0.

In [12]:
###############################
#  Question 2
###############################


def Question2(X, Y, init_weight, init_bias, learning_rate):
    model = MLP([784,10], relu, relu_gradient, categorical_cross_entropy,categorical_cross_entropy_prime, learning_rate, 1, 100,np.ones((Y_train.shape[0], 10)), init_weight, init_bias, optimizer = 'simple', beta = 0)
    return model.weights, model.bias
# this function returns the updated weights and biases
Question2(X,Y,weights, bias, 0.001)
weights, bias = Question2(X,Y,weights, bias, 0.001)

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Load the MNIST dataset
mnist_data = pd.read_csv("mnist_train.csv")

# Assuming the labels are in the first column and pixel values start from the second column
X = mnist_data.iloc[:, 1:]
Y = mnist_data.iloc[:, 0]

# Split the data into training and testing sets
# By default, the test_size is set to 0.25, meaning 25% of the data will be used for testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [20]:
def predict_class(input_vector):
    input_vector = np.array(input_vector).reshape(1, -1)
    logits = np.dot(input_vector, weights) + bias
    softmax_output = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    predicted_class = np.argmax(softmax_output, axis=1)[0]
    return predicted_class

# # Example usage
input_example = X_test[1004]  # Replace with the input vector you want to predict
predicted_class_example = predict_class(input_example)
print(f"Predicted Class: {predicted_class_example}")
print(Y_test[1004])

Predicted Class: [0 0 0 0 0 0 0 0 0 0]
6


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2148\2528251222.py:4: RuntimeWarning: overflow encountered in exp
  softmax_output = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_2148\2528251222.py:4: RuntimeWarning: invalid value encountered in divide
  softmax_output = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)


In [11]:
#### Splitting the data into test and train datasets

import pandas as pd
from sklearn.model_selection import train_test_split

# Load the MNIST dataset
mnist_data = pd.read_csv("mnist_train.csv")

# Assuming the labels are in the first column and pixel values start from the second column
X = mnist_data.iloc[:, 1:]
Y = mnist_data.iloc[:, 0]
# print(X)
# print(X.shape)
# print(Y)
# print(Y.shape)

# Split the data into training and testing sets
# By default, the test_size is set to 0.25, meaning 25% of the data will be used for testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

# Print the shapes of the resulting sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", Y_train.shape)
print("y_test shape:", Y_test.shape)
# print("X_test is : " ,X_test)
# print(X_test.shape)
# print("Y_test is : " , Y_test)
print(Y_test.shape)


MLP().accuracy(X_test,Y_test)

X_train shape: (45000, 784)
X_test shape: (15000, 784)
y_train shape: (45000,)
y_test shape: (15000,)
(15000,)


TypeError: MLP.__init__() missing 13 required positional arguments: 'network', 'activation_function', 'derivative_activation_function', 'loss_func', 'loss_func_derivative', 'learning_rate', 'epoch', 'diff_cost', 'Ypred', 'init_weights', 'init_bias', 'optimizer', and 'beta'

In [10]:
#####################
# Question 3
#####################

def Question3(X, Y, init_weight, init_bias, learning_rate):
    network_list = []
    for i in range(len(init_weight)):
        network_list.append(init_weight[i].shape[0])
    network_list.append(init_weight[-1].shape[1])
    model = MLP(network_list, relu, relu_gradient, categorical_cross_entropy,categorical_cross_entropy_prime, learning_rate, 1, 100,np.ones((Y_train.shape[0], 10)), init_weight, init_bias, optimizer = 'simple', beta = 0)
    return model.weights, model.bias


Question3(X,Y, weights, bias, 0.001)

([array([[ 0.00433276, -0.04921942,  0.07032036, ...,  0.04543782,
           0.00232675, -0.0492368 ],
         [-0.05681664, -0.00558825,  0.09881505, ...,  0.02941765,
           0.01717794,  0.01828841],
         [-0.00957505, -0.01864338, -0.08053478, ...,  0.02758056,
           0.06751622, -0.05917661],
         ...,
         [ 0.08745037,  0.00737087,  0.05731651, ..., -0.13073172,
          -0.0019095 , -0.05602664],
         [-0.01890735, -0.07924546, -0.05136649, ..., -0.07931404,
           0.00901902, -0.05785499],
         [ 0.05864962,  0.08517096,  0.02903769, ...,  0.06477124,
           0.02560436,  0.02573085]])],
 [array([[ 0.27980491, -0.23114314,  0.3057736 , -0.46551313,  0.64912974,
          -0.38258846,  0.03504389, -0.31666674,  0.37115068,  1.27388835]])])

In [11]:
######################
# Question 4
###################### 

def Question4(X, Y, init_weight, init_bias, learning_rate):
    network_list = []
    for i in range(len(init_weight)):
        network_list.append(init_weight[i].shape[0])
    network_list.append(init_weight[-1].shape[1])
    model = MLP(network_list, sigmoid, sigmoid_gradient, categorical_cross_entropy,categorical_cross_entropy_prime, learning_rate, 1, 100,np.ones((Y_train.shape[0], 10)), init_weight, init_bias, optimizer = 'simple', beta = 0)
    return model.weights, model.bias

Question4(X,Y, weights, bias, 0.001)


([array([[ 0.00433276, -0.04921942,  0.07032036, ...,  0.04543782,
           0.00232675, -0.0492368 ],
         [-0.05681664, -0.00558825,  0.09881505, ...,  0.02941765,
           0.01717794,  0.01828841],
         [-0.00957505, -0.01864338, -0.08053478, ...,  0.02758056,
           0.06751622, -0.05917661],
         ...,
         [ 0.08745037,  0.00737087,  0.05731651, ..., -0.13073172,
          -0.0019095 , -0.05602664],
         [-0.01890735, -0.07924546, -0.05136649, ..., -0.07931404,
           0.00901902, -0.05785499],
         [ 0.05864962,  0.08517096,  0.02903769, ...,  0.06477124,
           0.02560436,  0.02573085]])],
 [array([[ 0.27980491, -0.23114314,  0.3057736 , -0.46551313,  0.64912974,
          -0.38258846,  0.03504389, -0.31666674,  0.37115068,  1.27388835]])])

In [12]:
##################
# Question 5
##################

def Question5(X, Y, init_weight, init_bias, learning_rate, activation_function, activation_function_derivative):
    network_list = []
    for i in range(len(init_weight)):
        network_list.append(init_weight[i].shape[0])
    network_list.append(init_weight[-1].shape[1])
    model = MLP(network_list, activation_function, activation_function_derivative, categorical_cross_entropy,categorical_cross_entropy_prime, learning_rate, 1, 100,np.ones((Y_train.shape[0], 10)),init_weight, init_bias, optimizer = 'simple', beta = 0)
    return model.weights, model.bias

Question5(X,Y, weights, bias, 0.001, relu, relu_gradient)

([array([[ 0.00433276, -0.04921942,  0.07032036, ...,  0.04543782,
           0.00232675, -0.0492368 ],
         [-0.05681664, -0.00558825,  0.09881505, ...,  0.02941765,
           0.01717794,  0.01828841],
         [-0.00957505, -0.01864338, -0.08053478, ...,  0.02758056,
           0.06751622, -0.05917661],
         ...,
         [ 0.08745037,  0.00737087,  0.05731651, ..., -0.13073172,
          -0.0019095 , -0.05602664],
         [-0.01890735, -0.07924546, -0.05136649, ..., -0.07931404,
           0.00901902, -0.05785499],
         [ 0.05864962,  0.08517096,  0.02903769, ...,  0.06477124,
           0.02560436,  0.02573085]])],
 [array([[ 0.27980491, -0.23114314,  0.3057736 , -0.46551313,  0.64912974,
          -0.38258846,  0.03504389, -0.31666674,  0.37115068,  1.27388835]])])

In [13]:
########################
# Question 6
########################


def Question6(X, Y, init_weight, init_bias, learning_rate, activation_function, activation_function_derivative, optimizer):
    network_list = []
    for i in range(len(init_weight)):
        network_list.append(init_weight[i].shape[0])
    network_list.append(init_weight[-1].shape[1])
    print(network)
    model = MLP(network_list, activation_function, activation_function_derivative, categorical_cross_entropy,categorical_cross_entropy_prime, learning_rate, 1, 100,np.ones((Y_train.shape[0], 10)),init_weight, init_bias, optimizer = optimizer, beta = 0.9)
    return model.weights, model.bias

Question6(X,Y, weights, bias, 0.001, sigmoid, sigmoid_gradient, 0.8)

[784, 10]


([array([[ 0.00433276, -0.04921942,  0.07032036, ...,  0.04543782,
           0.00232675, -0.0492368 ],
         [-0.05681664, -0.00558825,  0.09881505, ...,  0.02941765,
           0.01717794,  0.01828841],
         [-0.00957505, -0.01864338, -0.08053478, ...,  0.02758056,
           0.06751622, -0.05917661],
         ...,
         [ 0.08745037,  0.00737087,  0.05731651, ..., -0.13073172,
          -0.0019095 , -0.05602664],
         [-0.01890735, -0.07924546, -0.05136649, ..., -0.07931404,
           0.00901902, -0.05785499],
         [ 0.05864962,  0.08517096,  0.02903769, ...,  0.06477124,
           0.02560436,  0.02573085]])],
 [array([[ 0.27980491, -0.23114314,  0.3057736 , -0.46551313,  0.64912974,
          -0.38258846,  0.03504389, -0.31666674,  0.37115068,  1.27388835]])])